### Dữ liệu FireAnt
---

#### Hằng số chung

In [ ]:
# Dãy Authenication Bearer Token cần sử dụng để gọi API
AUTH_BEARER = ''

# Dãy các key giá trị cần lấy từ API
USEFUL_KEYS = ['postID', 'date', 'postGroup', 'title', 'description']

#### Hàm gọi API

In [2]:
import requests
import json

def get_request(url, params=None):
    """
    Hàm gọi API chung tới FireAnt
    Trả về dữ liệu JSON nếu thành công, None nếu thất bại

    url: đường dẫn API, params: tham số gửi đi
    """

    headers = {
        'Authorization': f'Bearer {AUTH_BEARER}',
    }
    response = requests.get(url, headers=headers, params=params)
    if response.status_code != 200:
        print(f'u failed god: {response.status_code}')
        return None
    return response.json()

def get_posts(offset = 0, limit = 10):
    """
    Hàm lấy danh sách các bài viết

    offset: vị trí bắt đầu lấy, limit: số lượng bài viết cần lấy
    """

    url = 'https://api.fireant.vn/posts'
    params = {
        'type': 1,
        'offset': offset,
        'limit': limit,
    }
    return get_request(url, params)

def get_replies(post_id, offset = 0, limit = 10):
    """
    Hàm lấy danh sách các bình luận của một bài viết

    post_id: id của bài viết cần lấy, offset: vị trí bắt đầu lấy, limit: số lượng bình luận cần lấy
    """

    url = f'https://api.fireant.vn/posts/{post_id}/replies'
    params = {
        'offset': offset,
        'limit': limit,
    }
    return get_request(url, params)


#### Hàm xử lý dữ liệu bài viết

Hàm này là hàm helper, đầu vào là một bài viết đã được cào, để xử lý các key và bỏ các key không cần thiết.
Trả về bài viết đã được xử lý

In [14]:
def process_post(post: dict):
    """
    Hàm xử lý dữ liệu bài viết, xử lý qua các key và bỏ các key không cần thiết
    Trả về bài viết đã được xử lý
    """

    # Chỉ giữ lại các key cần thiết
    post = {k: v for k, v in post.items() if k in USEFUL_KEYS}
    
    # # Xử lý các mã được đề cập trong key 'taggedSymbols'
    # list_symbol = post['taggedSymbols']
    # list_processed_symbol = [] # Danh sách các mã sau khi xử lý

    # for symbol in list_symbol:
    #     # symb: mã cổ phiếu
    #     # price: giá cổ phiếu tại thời điểm bài viết được đăng
    #     try:
    #         list_processed_symbol.append({
    #             'symb': symbol['symbol'],
    #             'price': round(float(symbol['price']),2),
    #         })
    #     except:
    #         pass
    
    # post['taggedSymbols'] = json.dumps(list_processed_symbol) # Gán lại giá trị mới cho key 'taggedSymbols'

    # Xử lý các key cần thiết
    # post['userid'] = post['user']['id']
    # post['totalImages'] = len(post['images'])
    # post['totalFiles'] = len(post['files'])
    # post['totalSymbols'] = len(list_processed_symbol)
    
    # Xóa các key không cần thiết
    # del post['user']
    # del post['images']

    try:
        post['group'] = str(post['postGroup']['name'])
    except:
        post['group'] = "Không xác định"
    
    try:
        del post['postGroup']
    except:
        pass

    return post

#### Hàm cào bài viết

In [9]:
import dateutil.parser
from datetime import datetime
import csv

def crawl_posts(offset = 0, limit = 1000, number_of_entries = 26262, end_date_str = '2024-12-29T00:00:00+07:00'):
    """
    Hàm cào dữ liệu bài viết từ FireAnt

    offset: vị trí bắt đầu, limit: số lượng bài viết lấy một lúc, number_of_entries: số lượng bài viết tối đa cần lấy
    end_date_str: ngày kết thúc lấy dữ liệu
    """

    result_posts_id = {}        # Dict tổng hợp ID các bài viết đã lấy, tránh trùng lặp
    count_posts = 0             # Số lượng bài viết đã lấy
    count_api_call = 0          # Số lần gọi API

    # Lặp cho đến khi lấy đủ số lượng bài viết cần lấy, hoặc hết dữ liệu
    while count_posts < number_of_entries:
        count_api_call += 1
        data = get_posts(offset, limit) # Lấy dữ liệu từ API

        if not data: # Nếu dữ liệu rỗng, thoát vòng lặp
            break
        
        # Nếu thời gian của bài viết cuối cùng nhỏ hơn ngày end_date, thoát vòng lặp
        # Ta chỉ lấy dữ liệu đến ngày end_date
        cur_date = dateutil.parser.isoparse(data[-1]['date']) # Thời gian của bài viết cuối cùng
        end_date = dateutil.parser.isoparse(end_date_str) # Thời gian kết thúc lấy dữ liệu
        if cur_date < end_date:
            break
        
        count_overlap = 0 # Số lượng bài viết trùng lặp
        result_posts = [] # Danh sách bài viết cần lấy

        for post in data:
            # Nếu bài viết đã được lấy, tăng biến đếm và bỏ qua
            if post['postID'] in result_posts_id: 
                count_overlap += 1
                continue
            
            # Thêm ID bài viết vào danh sách đã lấy
            result_posts_id[post['postID']] = True 
            result_posts.append(process_post(post)) 
            count_posts += 1
        
        # Tính lại offset cho lần lấy tiếp theo
        offset += limit + count_overlap

        # Ghi dữ liệu vào file CSV
        keys = result_posts[0].keys() # Lấy danh sách key của dữ liệu
        with open('posts.csv', 'a', newline='', encoding='utf-8') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=keys) # Tạo writer
            if(count_api_call == 1): # Nếu là lần ghi đầu tiên, ghi header
                writer.writeheader()
            writer.writerows(result_posts) # Ghi dữ liệu vào file

        print(f'Call no. {count_api_call}, this time got: {len(result_posts)} posts, total crawled {count_posts} posts, offset: {offset}', end='\r')

    print(f'\nCrawled {count_posts} posts, end of data')

#### Hàm cào bình luận


In [6]:
import pandas as pd
from time import sleep as delay

def crawl_replies(offset = 0, limit = 1000, skip = 0, range_of_replies = (20, 1000)):
    """
    Hàm cào dữ liệu bình luận từ FireAnt, sử dụng danh sách bài viết `posts.csv`.

    offset: vị trí bắt đầu (bình luận), limit: số lượng bình luận lấy một lúc
    skip: số lượng bài viết ban đầu được bỏ qua
    range_of_replies: bài có số lượng bình luận nằm trong khoảng này mới được lấy
    """

    df = pd.read_csv('posts.csv')   # Dataframe chính từ danh sách bài viết trong file CSV
    count_replies = 0               # Số lượng bình luận đã lấy

    # Lọc các bài viết có số lượng bình luận nằm trong khoảng range_of_replies
    post_list = df[(df['totalReplies'] >= range_of_replies[0]) & (df['totalReplies'] <= range_of_replies[1])]['postID'].tolist()
    print(f'Found {len(post_list)} posts with totalReplies between {range_of_replies[0]} and {range_of_replies[1]}')

    # Xét từng bài viết
    for post_idx, postID in enumerate(post_list):
        if post_idx < skip: # Bỏ qua số lượng bài viết ban đầu
            continue

        # Thử lấy dữ liệu bình luận cho bài viết, tối đa 4 lần
        data = get_replies(postID, offset, limit)
        if not data: 
            # Nếu không lấy được dữ liệu, thử lại 3 lần, mỗi lần delay 3 giây
            for retry in range(3):
                print(f'Failed to get replies for post no. {postID} ({post_idx+1}/{len(post_list)}), retrying... ({retry+1}/3)', end='\r')
                data = get_replies(postID, offset, limit)
                if data: # Nếu lấy được dữ liệu, thoát vòng lặp
                    break 
                delay(3)

        # Nếu vẫn không lấy được dữ liệu, bỏ qua bài viết
        if not data:
            print(f'Failed to get replies for post no. {postID} ({post_idx+1}/{len(post_list)}), skipping...')
            continue

        result_replies = [] # Danh sách bình luận cần lấy

        # Xử lý dữ liệu bình luận
        for post in data:
            result_replies.append(process_post(post))
            count_replies += 1

        # Ghi dữ liệu vào file CSV
        keys = result_replies[0].keys() # Lấy danh sách key của dữ liệu
        with open('replies.csv', 'a', newline='', encoding='utf-8') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=keys) # Tạo writer
            if(post_idx == 0): # Nếu là lần ghi đầu tiên, ghi header
                writer.writeheader()
            writer.writerows(result_replies) # Ghi dữ liệu vào file
        
        print(f'Crawled post no. {postID} ({post_idx+1}/{len(post_list)}), got {len(result_replies)} replies, total crawled {count_replies} replies', end='\r')
        delay(0.2) # Delay 0.2 giây giữa các bài viết, tránh bị block
    
    print(f'\nCrawled {len(post_list)} posts, end of data')

#### Hàm làm sạch dữ liệu
Hàm này để làm sạch dữ liệu trong các file CSV.

In [20]:
import pandas as pd

def clean_data(csvfile):
    """
    Hàm xử lý dữ liệu sau khi cào, xóa các bài viết trùng lặp và các cột không cần thiết

    csvfile: file CSV cần xử lý
    """

    # Đọc file CSV
    df = pd.read_csv(csvfile) 

    # Xóa các bài viết trùng lặp
    df = df.drop_duplicates(subset=['postID'])

    # Ghi dữ liệu vào file CSV mới (cleaned_{csvfile})
    df.to_csv(f'cleaned_{csvfile}', index=False)

    print(f'Cleaned data saved to cleaned_{csvfile}, total {len(df)} entries')

---
### Thực thi

In [17]:
# Cào dữ liệu bài viết
crawl_posts(limit=1000)

Call no. 24, this time got: 1000 posts, total crawled 24000 posts, offset: 24000
Crawled 24000 posts, end of data


In [21]:
# Làm sạch dữ liệu trong 2 file CSV
clean_data('posts.csv')

Cleaned data saved to cleaned_posts.csv, total 24000 entries


In [3]:
import pandas as pd
df=pd.read_csv('cleaned_posts.csv')
df = df[df['group'] != "Doanh nghiệp"]
print(len(df))

18983
